In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a"], keep="first")
print(len(df))

716107
705250


In [38]:
wikid = pl.read_parquet(os.path.join('data', 'wikidata.parquet'))
wikid.sample(5)

024_a,ceny,druh_umrti,facebook,instagram,jazykove_verze,label_cs,label_en,manzelstvo,obcanstvi,partnerstvo,popis_cs,popis_en,potomstvo,pricina_umrti,profese,role,skoly,sourozenectvo,strany,twitter,udalosti,vezeni,w_gender,w_misto_narozeni,w_misto_umrti,w_narozeni,w_narozeni_presne,w_umrti,w_umrti_presne,web,wiki_cs,wiki_en,__index_level_0__
str,list[str],str,str,str,list[str],str,str,list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],str,list[str],list[str],f64,str,f64,str,str,str,str,str
"""Q18410054""","[""Heinrich Maria Ledig-Rowohlt Award""]",null,null,null,"[""de""]","""Werner Schmitz""","""Werner Schmitz""",null,"[""Německo""]",null,null,"""German translator""",null,null,"[""překladatel""]",null,null,null,null,null,null,null,"""muž""","[""Kolín nad Rýnem""]",null,1953.0,"""+1953-11-20T00:00:00Z""",null,null,null,null,null,"""xx0318933"""
"""Q120575071""",null,null,null,null,[],"""Jana Kolaříková""",null,null,null,null,"""autorka knih a učebnic o výživ…",null,null,null,null,null,null,null,null,null,null,null,"""žena""",null,null,null,null,null,null,null,null,null,"""xx0015531"""
"""Q1745663""",null,null,null,null,"[""de""]","""Klaus Schmidt""","""Klaus Schmidt""",null,"[""Německo""]",null,null,"""German author and historian""",null,null,"[""historik"", ""spisovatel""]",null,"[""Univerzita v Bonnu""]",null,null,null,null,null,"""muž""","[""Rheydt""]",null,1935.0,"""+1935-12-09T00:00:00Z""",null,null,null,null,null,"""skuk0005082"""
"""Q95139072""",null,null,null,null,[],"""Edita Kotzianová""","""Edita Kotzianová""",null,"[""Československo""]",null,"""Narozena 27. 11. 1928 v Brně. …",null,null,null,"[""choreograf"", ""pedagog"", ""taneční mistr""]",null,null,null,null,null,null,null,"""žena""","[""Brno""]","[""Brno""]",1928.0,"""+1928-11-27T00:00:00Z""",2020.0,"""+2020-10-16T00:00:00Z""",null,null,null,"""jo2005321091"""
"""Q95214150""",null,null,null,null,[],"""Otto Oliva""","""Otto Oliva""",null,null,null,"""Narozen 1.1.1889 v Mezilesí u …",null,null,null,"[""lektor"", ""učitel"", ""spisovatel""]",null,"[""Filozofická fakulta Univerzity Karlovy""]",null,null,null,null,null,"""muž""","[""Mezilesí""]","[""Praha""]",1889.0,"""+1889-01-01T00:00:00Z""",1970.0,"""+1970-02-20T00:00:00Z""",null,null,null,"""jk01091048"""


In [60]:
narozeni_wikid = wikid.explode('w_misto_narozeni').rename({'__index_level_0__':'100_7','w_misto_narozeni':'rodna_obec'}).select(pl.col(['100_7','rodna_obec'])).drop_nulls()
narozeni_wikid.sample(5)

100_7,rodna_obec
str,str
"""jk01022465""","""Lužice"""
"""jn20000700471""","""East Windsor"""
"""jn20000401069""","""Olomouc"""
"""jk01130278""","""Zlín"""
"""jk01110171""","""Chlumec nad Cidlinou"""


In [4]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(pl.col("100_7")).to_series().to_list()
len(cesi)

363993

In [7]:
df = df.filter(pl.col("100_7").is_in(cesi))

In [24]:
aut.explode("100_a").filter(pl.col("100_a").str.contains("Němcová, Bo"))

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001
list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str
"[""isni"", ""wikidata""]","[""0000000121343289"", ""Q157970""]","[""1820""]","[""1862""]","[""jk01083016""]","""Němcová, Božena,""","[""1820-1862""]","[""1""]",null,"[""Vídeň, Rakousko""]","[""Praha, Česko""]","[""Česko""]",null,"[""česká literatura"", ""národní obrození"", … ""české pohádky""]",null,"[""spisovatelky"", ""publicistky""]","[""žena""]","[""cze""]","[""Novotná, Barbora,"", ""Panklová, Barbora,"", … ""Nímtsúfá, Búdžíná,""]","[""1820-1862"", ""1820-1862"", … ""1820-1862""]","[""Rodné jméno:"", null, … null]","[""1"", ""1"", … ""1""]",null,null,null,null,null,null,null,null,"[""Narozena zřejmě 4. 2. 1820 ve Vídni, zemřela 21. 1. 1862 v Praze. Spisovatelka, publicistka, autorka a sběratelka pohádek.""]",null,""" cz a22 n 4500""","""jk01083016"""
null,null,"[""1879""]","[""1949""]","[""jk01083017""]","""Němcová, Božena,""","[""1879-1949""]","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,"[""Oborská, Božena,""]","[""1879-1949""]","[""Pseudonym:""]","[""1""]",null,null,null,null,null,null,null,null,"[""Narozena 1879, zemřela 26.2.1949 v Praze. Přírodovědecké články a překlady.""]",null,""" nz a22 n 4500""","""jk01083017"""
null,null,null,null,"[""aun2006373269""]","""Němcová, Božena,""","[""1820-1862.""]","[""1""]",null,null,null,null,null,null,null,null,null,null,"[""Němcová, Božena,""]","[""1820-1862.""]",null,"[""1""]",null,null,null,null,null,null,null,null,null,null,""" cz a22 n 4500""","""aun2006373269"""
null,null,null,null,"[""aun2006373270""]","""Němcová, Božena,""","[""1820-1862.""]","[""1""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,""" nz a22 n 4500""","""aun2006373270"""
null,null,null,null,"[""aun2007402293""]","""Němcová, Božena,""","[""1820-1862.""]","[""1""]",null,null,null,null,null,null,null,null,null,null,"[""Němcová, Božena,""]","[""1820-1862.""]",null,"[""1""]",null,null,null,null,null,null,null,null,null,null,""" cz a22 n 4500""","""aun2007402293"""
null,null,null,null,"[""aun2008457809""]","""Němcová, Božena,""","[""1820-1862.""]","[""1""]",null,null,null,null,null,null,null,null,null,null,"[""Němcová, Božena,""]","[""1820-1862.""]",null,"[""1""]",null,null,null,null,null,null,null,null,null,null,""" nz a22 n 4500""","""aun2008457809"""
null,null,null,null,"[""aun2008461060""]","""Němcová, Božena,""","[""1820-1862.""]","[""1""]",null,null,null,null,null,null,null,null,null,null,"[""Němcová, Božena,"", ""Němcová, Božena,""]","[""1820-1862."", ""1820-1862.""]",null,"[""1"", ""1""]",null,null,null,null,null,null,null,null,null,"[""https://www.muzeumbn.cz/transbabicka/""]",""" cz a22 n 4500""","""aun2008461060"""
null,null,null,null,"[""aun2008461061""]","""Němcová, Božena,""","[""1820-1862.""]","[""1""]",null,null,null,null,null,null,null,null,null,null,"[""Němcová, Božena,""]","[""1820-1862.""]",null,"[""1""]",null,null,null,null,null,null,null,null,null,null,""" nz a22 n 4500""","""aun2008461061"""
null,null,null,null,"[""aun2008462806""]","""Němcová, Božena,""","[""1820-1862.""]","[""1""]",null,null,null,null,null,null,null,null,null,null,"[""Němcová, Božena,"", ""Němcová, Božena,""]","[""1820-1862."", ""1820-1862.""]",null,"[""1"", ""1""]",null,null,null,null,null,null,null,null,null,null,""" nz a22 n 4500""","""aun2008462806"""


In [50]:
narozeni_aut = aut.explode('100_7').explode("370_a").drop_nulls(subset=['370_a']).with_columns(pl.col("370_a").map_elements(lambda x: x.split(",")[0] if "," in x else None).alias("rodna_obec")).select(pl.col(['100_7','rodna_obec']))
narozeni_aut

100_7,rodna_obec
str,str
"""jk01010004""","""Odrlice"""
"""jk01010022""","""Příbram"""
"""jk01010030""","""Praha"""
"""jk01010034""","""Štítina"""
"""jk01010104""","""Praha"""
"""jk01010058""","""Praha"""
"""jk01010070""","""Bludov"""
"""jk01010073""","""Plzeň"""
"""jk01010087""","""Hlinsko"""


In [56]:
def specialized_join(df1: pl.DataFrame, df2: pl.DataFrame) -> pl.DataFrame:
    # Perform outer join
    joined = df1.join(
        df2,
        on="100_7",
        how="outer",
        suffix="_df2"
    )
    
    # Coalesce all columns, including the ID column
    result = joined.with_columns([
        pl.coalesce("100_7", "100_7_df2").alias("100_7"),
        pl.coalesce("rodna_obec", "rodna_obec_df2").alias("rodna_obec")
    ])
    
    # Drop the duplicate columns
    result = result.drop(["100_7_df2", "rodna_obec_df2"])
    
    return result

In [62]:
specialized_join(narozeni_aut, narozeni_wikid)

100_7,rodna_obec
str,str
"""jk01010005""","""Boskovice"""
"""jk01010022""","""Příbram"""
"""jk01010023""","""Pardubice"""
"""jk01010026""","""Loštice"""
"""jk01010030""","""Praha"""
"""jk01010034""","""Štítina"""
"""jk01010036""","""Nemojany"""
"""jk01010040""","""Nemojany"""
"""jk01010048""","""Frenštát pod Radhoštěm"""


In [64]:
rodne_obce = specialized_join(narozeni_aut, narozeni_wikid)

In [66]:
df = df.join(rodne_obce, on='100_7', how='left')

In [70]:
df.group_by('rodna_obec').len().sort(by='len',descending=True).head(10)

rodna_obec,len
str,u32
null,160063
"""Praha""",77745
"""Brno""",15360
"""Plzeň""",6894
"""Ostrava""",4273
"""Olomouc""",4224
"""České Budějovice""",3701
"""Hradec Králové""",3652
"""Kladno""",2793


## Velikost a umístění měst

In [10]:
import requests
from bs4 import BeautifulSoup

In [12]:
r = requests.get("https://cs.wikipedia.org/wiki/Seznam_m%C4%9Bst_v_%C4%8Cesku_podle_po%C4%8Dtu_obyvatel")
soup = BeautifulSoup(r.text, 'html.parser')
mesta = {}
tabulka = soup.find(class_="sortable")
for radek in tabulka.find_all('tr'):
    if 'Rozloha' not in radek.text:
        mesta[radek.find_all('td')[1].text.strip()] = int(''.join([x for x in radek.find_all('td')[3].text.strip() if x.isdigit()]))
print(len(mesta))
r = requests.get("https://cs.wikipedia.org/wiki/Seznam_nejlidnat%C4%9Bj%C5%A1%C3%ADch_obc%C3%AD_v_%C4%8Cesku_bez_statusu_m%C4%9Bsta")
soup = BeautifulSoup(r.text, 'html.parser')
vsi = {}
tabulka = soup.find(class_="sortable")
for radek in tabulka.find_all('tr'):
    try:
        if 'Početoby' not in radek.text:
            ves = radek.find_all('td')[0].text.strip()
            velikost = int(''.join([x for x in radek.find_all('td')[1].text.strip() if x.isdigit()]))
            vsi[ves] = velikost
    except:
        print(radek.text)
len(vsi)
obce = pd.concat([pd.Series(mesta), pd.Series(vsi)])
vsechny_obce = obce.index.to_list()
len(vsechny_obce)

607


707

In [14]:
lokality = pd.read_json(os.path.join('../ceske-obce/ceske-obce.json'))

In [16]:
lokality

,name,wikidata_id,coordinates,population,area,region,district
1097,Praha,Q1085,Point(14.421388888 50.0875),1384732,496.210000,Czech Republic,Unknown
1143,Brno,Q14960,Point(16.608333333 49.195277777),400566,230.182739,Brno-City District,Unknown
1136,Ostrava,Q8385,Point(18.2925 49.835555555),284765,214.229602,Ostrava-City District,Unknown
1086,Plzeň,Q43453,Point(13.3825 49.741388888),185599,137.671042,Plzeň-City District,Unknown
952,Liberec,Q146351,Point(15.05 50.766666666),107982,106.087154,Liberec District,Unknown
...,...,...,...,...,...,...,...
526,Hill fort Chlum,Q110290703,Point(14.926382303 50.378408905),0,0.000000,Nepřevázka,Unknown
273,Q37453718,Q37453718,Point(13.8794367 49.0689152),0,0.000000,Lštění,Unknown
527,Q47035482,Q47035482,Point(13.51383 49.7611342),0,0.000000,Dýšina,Unknown
528,Q106630428,Q106630428,Point(14.5871 50.0557),0,0.000000,Dubeč,Unknown


## Různé pohledy

In [157]:
do_10000 = pl.from_pandas(lokality).filter(pl.col('population').is_between(5,10000)).select(pl.col('name')).to_series().to_list()
do_1000 = pl.from_pandas(lokality).filter(pl.col('population').is_between(5,1000)).select(pl.col('name')).to_series().to_list()
nad_10000 = pl.from_pandas(lokality).filter(pl.col('population') > 10000).select(pl.col('name')).to_series().to_list()
nad_1000 = pl.from_pandas(lokality).filter(pl.col('population') > 1000).select(pl.col('name')).to_series().to_list()
nad_5000 = pl.from_pandas(lokality).filter(pl.col('population') > 5000).select(pl.col('name')).to_series().to_list()

### Knih na obec

In [163]:
df.group_by("rodna_obec").len().sort(by='len',descending=True).head(100)

rodna_obec,len
str,u32
null,160063
"""Praha""",77745
"""Brno""",15360
"""Plzeň""",6894
"""Ostrava""",4273
"""Olomouc""",4224
"""České Budějovice""",3701
"""Hradec Králové""",3652
"""Kladno""",2793


### Rodáků na obec (neřeší se hody jmen)

In [167]:
df.group_by("rodna_obec").agg(pl.col('100_a').unique().len()).sort(by='100_a',descending=True).head(100)

rodna_obec,100_a
str,u32
null,53028
"""Praha""",8437
"""Brno""",2160
"""Plzeň""",940
"""Ostrava""",765
"""Olomouc""",558
"""České Budějovice""",518
"""Hradec Králové""",485
"""Pardubice""",424


Videň a Bratislava víc než Karlovy Vary.

### Knih na č. obec do 10000 obyvatel (Vídeň je nesmysl)

In [147]:
df.filter(pl.col('rodna_obec').is_in(do_10000)).group_by('rodna_obec').len().sort(by='len',descending=True).head(10)

rodna_obec,len
str,u32
"""Vídeň""",2466
"""Hronov""",1410
"""Hořice""",1113
"""Polička""",918
"""Malé Svatoňovice""",867
"""Nové Město na Moravě""",866
"""Nivnice""",767
"""Ivančice""",665
"""Chlumec nad Cidlinou""",625


### Rodáků na obec do 10k

In [145]:
df.filter(pl.col('rodna_obec').is_in(do_10000)).group_by('rodna_obec').agg(pl.col('100_a').unique().len()).sort(by='100_a',descending=True).head(10)

rodna_obec,100_a
str,u32
"""Vídeň""",194
"""Nové Město na Moravě""",155
"""Polička""",100
"""Hořice""",81
"""Košice""",78
"""Jilemnice""",77
"""Ivančice""",75
"""Dačice""",71
"""Chlumec nad Cidlinou""",56


In [143]:
df.filter(pl.col('rodna_obec').is_in(do_1000)).group_by('rodna_obec').len().sort(by='len',descending=True).head(10)

rodna_obec,len
str,u32
"""Vídeň""",2466
"""Miletín""",594
"""Havlíčkova Borová""",575
"""Košice""",517
"""Karlín""",508
"""Nové Město""",422
"""Trhové Dušníky""",392
"""Hrusice""",383
"""Josefov""",360


In [141]:
df.filter(pl.col('rodna_obec').is_in(do_1000)).group_by('rodna_obec').agg(pl.col('100_7').unique().len()).sort(by='100_7',descending=True).head(10)

rodna_obec,100_7
str,u32
"""Vídeň""",195
"""Košice""",78
"""Nové Město""",33
"""Karlín""",32
"""Žilina""",22
"""Vítkovice""",20
"""Josefov""",18
"""Sedlec""",14
"""Komárov""",13


In [137]:
rodactvo = df.group_by('rodna_obec').agg(pl.col('100_7').unique().len()).sort(by='100_7',descending=True).join(pl.from_pandas(lokality).select(pl.col(['name','population'])), left_on='rodna_obec', right_on='name').with_columns((pl.col('100_7') / pl.col('population')).alias('podil')).sort(by='podil',descending=True)

In [139]:
rodactvo.filter(pl.col('rodna_obec').is_in(do_10000))

rodna_obec,100_7,population,podil
str,u32,i64,f64
"""Vídeň""",195,456,0.427632
"""Nové Zámky""",6,30,0.2
"""Karlín""",32,189,0.169312
"""Košice""",78,782,0.099744
"""Nové Město""",33,402,0.08209
"""Minice""",2,28,0.071429
"""Útěchovice""",4,61,0.065574
"""Nový Dvůr""",4,67,0.059701
"""Čilá""",1,18,0.055556


In [151]:
rodactvo.filter(pl.col('rodna_obec').is_in(nad_10000))

rodna_obec,100_7,population,podil
str,u32,i64,f64
"""Litomyšl""",151,10493,0.014391
"""Jičín""",195,16230,0.012015
"""Čáslav""",123,10512,0.011701
"""Rychnov nad Kněžnou""",131,11442,0.011449
"""Kyjov""",114,10799,0.010557
"""Vysoké Mýto""",123,12412,0.00991
"""Boskovice""",119,12190,0.009762
"""Turnov""",139,14502,0.009585
"""Kutná Hora""",202,21556,0.009371


In [155]:
rodactvo.filter(pl.col('rodna_obec').is_in(nad_1000))

rodna_obec,100_7,population,podil
str,u32,i64,f64
"""Přílepy""",18,1014,0.017751
"""Čeladná""",51,2904,0.017562
"""Opočno""",54,3140,0.017197
"""Kněževes""",17,1018,0.016699
"""Nové Město na Moravě""",155,9904,0.01565
"""Počátky""",38,2500,0.0152
"""Městec Králové""",42,2860,0.014685
"""Litomyšl""",151,10493,0.014391
"""Jilemnice""",78,5443,0.01433


In [159]:
rodactvo.filter(pl.col('rodna_obec').is_in(nad_5000))

rodna_obec,100_7,population,podil
str,u32,i64,f64
"""Nové Město na Moravě""",155,9904,0.01565
"""Litomyšl""",151,10493,0.014391
"""Jilemnice""",78,5443,0.01433
"""Jičín""",195,16230,0.012015
"""Čáslav""",123,10512,0.011701
"""Rychnov nad Kněžnou""",131,11442,0.011449
"""Polička""",100,8939,0.011187
"""Kyjov""",114,10799,0.010557
"""Chlumec nad Cidlinou""",56,5592,0.010014


### Poměr k obyvatelstvu

In [219]:
final = df.group_by("rodna_obec").len().rename({'len':'knih'}).join(
    df.group_by('rodna_obec').agg(pl.col('100_a').unique().len()).rename({'100_a':'rodaku'}), on='rodna_obec', how='left').join(
    pl.from_pandas(lokality).select(pl.col(['name','population'])), left_on='rodna_obec', right_on='name', how='left'
).rename({'population':'obyvatel'}
).with_columns(
    (pl.col('rodaku') / pl.col('obyvatel')).alias('rodaku/obyvatel')
).with_columns((pl.col('knih') / pl.col('obyvatel')).alias('knih/obyvatel'))
final

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Chrást""",76,6,1923,0.00312,0.039522
"""Tartu""",12,1,null,null,null
"""Kazaň""",1,1,null,null,null
"""Račiněves""",15,1,null,null,null
"""Požáry""",13,1,null,null,null
"""Litoměřice""",336,84,22983,0.003655,0.01462
"""Mělník""",661,89,20350,0.004373,0.032482
"""Verona""",1,1,null,null,null
"""Budyně nad Ohří""",59,11,2174,0.00506,0.027139


In [231]:
final.filter(pl.col('obyvatel') > 0).drop_nulls().sort(by='knih/obyvatel',descending=True)

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Vídeň""",2466,194,456,0.425439,5.407895
"""Karlín""",508,32,189,0.169312,2.687831
"""Biskoupky""",276,2,188,0.010638,1.468085
"""Prádlo""",322,1,222,0.004505,1.45045
"""Dědice""",174,4,135,0.02963,1.288889
"""Útěchovice""",76,4,61,0.065574,1.245902
"""Benátky""",144,4,122,0.032787,1.180328
"""Třebíz""",279,3,240,0.0125,1.1625
"""Nové Město""",422,33,402,0.08209,1.049751


In [233]:
final.filter(pl.col('obyvatel') > 0).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Vídeň""",2466,194,456,0.425439,5.407895
"""Nové Zámky""",18,6,30,0.2,0.6
"""Karlín""",508,32,189,0.169312,2.687831
"""Košice""",517,78,782,0.099744,0.661125
"""Nové Město""",422,33,402,0.08209,1.049751
"""Minice""",20,2,28,0.071429,0.714286
"""Útěchovice""",76,4,61,0.065574,1.245902
"""Nový Dvůr""",47,4,67,0.059701,0.701493
"""Čilá""",7,1,18,0.055556,0.388889


In [257]:
final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='knih/obyvatel',descending=True)

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Malé Svatoňovice""",867,7,1591,0.0044,0.54494
"""Klenčí pod Čerchovem""",606,9,1459,0.006169,0.415353
"""Jimramov""",472,12,1207,0.009942,0.391052
"""Hronov""",1410,23,6096,0.003773,0.231299
"""Nivnice""",767,5,3373,0.001482,0.227394
"""Studená""",469,3,2205,0.001361,0.212698
"""Kněževes""",216,17,1018,0.016699,0.212181
"""Nové Hrady""",501,12,2506,0.004789,0.19992
"""Kožlany""",275,9,1546,0.005821,0.177878


In [259]:
final.filter(pl.col('obyvatel') > 10000).drop_nulls().sort(by='knih/obyvatel',descending=True)

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Litomyšl""",1530,151,10493,0.014391,0.145811
"""Čáslav""",1231,122,10512,0.011606,0.117104
"""Jičín""",1772,195,16230,0.012015,0.109181
"""Kutná Hora""",2161,202,21556,0.009371,0.100251
"""Boskovice""",1107,119,12190,0.009762,0.090812
"""Vysoké Mýto""",1092,123,12412,0.00991,0.087979
"""Rychnov nad Kněžnou""",1002,131,11442,0.011449,0.087572
"""Jaroměř""",1039,101,12541,0.008054,0.082848
"""Písek""",2567,256,30986,0.008262,0.082844


In [261]:
final.filter(pl.col('obyvatel') > 1000).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Přílepy""",140,18,1014,0.017751,0.138067
"""Čeladná""",235,51,2904,0.017562,0.080923
"""Opočno""",536,54,3140,0.017197,0.170701
"""Kněževes""",216,17,1018,0.016699,0.212181
"""Nové Město na Moravě""",866,155,9904,0.01565,0.087439
"""Počátky""",399,38,2500,0.0152,0.1596
"""Městec Králové""",242,42,2860,0.014685,0.084615
"""Litomyšl""",1530,151,10493,0.014391,0.145811
"""Vysoké nad Jizerou""",188,19,1329,0.014296,0.14146


In [263]:
final.filter(pl.col('obyvatel') > 10000).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Litomyšl""",1530,151,10493,0.014391,0.145811
"""Jičín""",1772,195,16230,0.012015,0.109181
"""Čáslav""",1231,122,10512,0.011606,0.117104
"""Rychnov nad Kněžnou""",1002,131,11442,0.011449,0.087572
"""Kyjov""",737,114,10799,0.010557,0.068247
"""Vysoké Mýto""",1092,123,12412,0.00991,0.087979
"""Boskovice""",1107,119,12190,0.009762,0.090812
"""Turnov""",1175,138,14502,0.009516,0.081023
"""Kutná Hora""",2161,202,21556,0.009371,0.100251


In [237]:
final.filter(pl.col('obyvatel') > 50000).drop_nulls().sort(by='rodaku/obyvatel',descending=True)

rodna_obec,knih,rodaku,obyvatel,rodaku/obyvatel,knih/obyvatel
str,u32,u32,i64,f64,f64
"""Praha""",77745,8437,1384732,0.006093,0.056144
"""Olomouc""",4224,558,102293,0.005455,0.041293
"""Brno""",15360,2160,400566,0.005392,0.038346
"""České Budějovice""",3701,518,97377,0.00532,0.038007
"""Zlín""",2033,384,74255,0.005171,0.027379
"""Hradec Králové""",3652,485,93906,0.005165,0.03889
"""Plzeň""",6894,940,185599,0.005065,0.037145
"""Pardubice""",2750,424,92362,0.004591,0.029774
"""Opava""",1532,252,55600,0.004532,0.027554
